# Linear roll motion analytical solution
As decribed by Journee, Peter Piel etc.

In [ ]:
import sympy as sp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sympy.physics.mechanics import dynamicsymbols

from rolldecay.symbols import *

from rolldecayestimators.substitute_dynamic_symbols import lambdify
from rolldecayestimators import direct_linear_estimator

from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex

In [ ]:
delta = sp.symbols('delta')
y = dynamicsymbols('y')
y0 = dynamicsymbols('y0')
y0_dot = y0.diff()
y0_dotdot = y0_dot.diff()

D = sp.symbols('D')



## General Linear ordinary differential equation:

In [ ]:
diff_eq = sp.Eq(y.diff().diff() + 2*delta*omega0*y.diff() + omega0**2*y,0)
Math(vlatex(diff_eq))

In [ ]:
equation_D = sp.Eq(D,sp.sqrt(1-delta**2))
equation_D

In [ ]:
lhs = y
rhs = sp.exp(-delta*omega0*t)*(y0*sp.cos(omega0*D*t) + (y0_dot/(omega0*D) + delta*y0/D)*sp.sin(omega0*D*t))

analytical_solution_general = sp.Eq(lhs,rhs)
Math(vlatex(analytical_solution_general))

In [ ]:
phi_0 = dynamicsymbols('phi_0')
phi_0_dot = phi_0.diff()
phi_0_dotdot = phi_0_dot.diff()


subs = [
    (y,phi),
    (y0, phi_0),
    (y0_dot, phi_0_dot),
    (y0_dotdot, phi_0_dotdot),
    (D,sp.solve(equation_D,D)[0]),
    (delta, zeta),
]

analytical_solution = analytical_solution_general.subs(subs)
Math(vlatex(analytical_solution))

In [ ]:
analytical_phi1d = sp.Eq(phi_dot,sp.simplify(analytical_solution.rhs.diff(t)))
Math(vlatex(analytical_phi1d))

In [ ]:
analytical_phi2d = sp.Eq(phi_dot_dot,sp.simplify(analytical_phi1d.rhs.diff(t)))
Math(vlatex(analytical_phi2d))

In [ ]:
analytical_solution_lambda = lambdify(sp.solve(analytical_solution,phi)[0])
analytical_solution_phi1d_lambda = lambdify(sp.solve(analytical_phi1d,phi_dot)[0])
analytical_solution_phi2s_lambda = lambdify(sp.solve(analytical_phi2d,phi_dot_dot)[0])


In [ ]:
time = np.linspace(0,100,1000)
df = pd.DataFrame(index=time)
df['t']=time
df['zeta'] = 0.1
df['omega0'] = 0.5
df['phi_0'] = np.deg2rad(10)
df['phi_01d'] = 0.0

result = df.copy()
result['phi'] = analytical_solution_lambda(**df,)
result['phi1d'] = analytical_solution_phi1d_lambda(**df,)
result['phi2d'] = analytical_solution_phi2s_lambda(**df,)

### Numerical solution

In [ ]:
linear_estimator = direct_linear_estimator.DirectLinearEstimator()

s0 = df.iloc[0]
time=df['t']
result_sim = linear_estimator.simulate(t=time,phi0=s0['phi_0'],phi1d0=s0['phi_01d'], omega0=s0['omega0'], zeta=s0['zeta'])
result_sim = result_sim.combine_first(df)

In [ ]:
fig,axes = plt.subplots(nrows = 3)
fig.set_size_inches(15,10)
keys = ['phi','phi1d','phi2d']
labels = ['$\Phi$ [rad]','$\dot{\Phi}$ [rad/s]','$\ddot{\Phi}$ $[rad/s^2]$']
for ax, key, label in zip(axes,keys, labels):
    result.plot(y=key, ax=ax, label='analytical')
    result_sim.plot(y=key, ax=ax, label='numerical',style='--')
    
    
    ax.set_ylabel(label)
    
ax.set_xlabel('Time [s]');
axes[0].set_title('Solutions to linear roll motion ($\omega_0$:%0.1f [rad/s], $\zeta$:%0.1f [-])' % 
             (df.iloc[0]['omega0'], df.iloc[0]['zeta']));



In [ ]:
from ipywidgets import interactive

In [ ]:
time = np.linspace(0,100,300)
df = pd.DataFrame(index=time)
df['t']=time

In [ ]:
def f(zeta=0.1,omega0=0.5,phi_0=10):
    fig,ax = plt.subplots()
    ax.set_xlim(0,time[-1])
    ax.set_ylim(-10,10)
    fig.set_size_inches(15,5)
    
    df['zeta'] = zeta
    df['omega0'] = omega0
    df['phi_0'] = np.deg2rad(phi_0)
    df['phi_01d'] = 0

    result = df.copy()
    result['phi'] = analytical_solution_lambda(**df,)
    result['phi']=np.rad2deg(result['phi'])
    ax.set_ylabel('Roll angle $\Phi$ [deg]')
    ax.set_xlabel('Time [s]')
    result.plot(y='phi', ax=ax)
    
    plt.show()

interactive_plot = interactive(f, zeta=(0.05,0.30, 0.01), omega0=(0.1,1,0.05), phi_0=(1.0,10.0,0.5))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot